### import module

In [1]:
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt

from mrmr import mrmr_classif

import numpy as np

import os

from lightgbm import LGBMClassifier

import pandas as pd
from pathlib import Path

import random

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

import seaborn as sns

from sklearn.calibration import CalibrationDisplay
from sklearn.cluster import KMeans
from sklearn.experimental import enable_halving_search_cv
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import (
    GroupKFold, GridSearchCV,
    KFold, 
    RandomizedSearchCV,
    StratifiedKFold, 
    train_test_split,
)

from tqdm import tqdm

### setup csv file path

In [2]:
BASE_DIR = Path(__name__).resolve().parent.parent
BASE_FILE_PATH = BASE_DIR / 'tabular-playground-series-nov-2022'
SUBMISSION_FILES_PATH = BASE_FILE_PATH / 'submission_files'

### code transcription(TPS Nov 2022 | EDA + Hybrid Stacking 🚀)

In [3]:
# SEED

seed = 2484
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)

#### Files
- submission_files/ - a folder containing binary model predictions
- train_labels.csv - the ground truth labels for the first half of the rows in the submission files
- sample_submission.csv - a sample submission file in the correct format, only containing the row ids for the second - - half of each file in the submissions folder; your task is to blend together submissions that achieve the improvements in the score.

In [4]:
submission = pd.read_csv(BASE_FILE_PATH / 'sample_submission.csv', index_col='id')

labels = pd.read_csv(BASE_FILE_PATH / 'train_labels.csv', index_col='id')

# the ids of the submission rows (useful later)
sub_ids = submission.index

# the ids of the labeled rows (useful later)
gt_ids = labels.index 

# list of files in the submission folder
subs = sorted(os.listdir(SUBMISSION_FILES_PATH))

#### Labels distribution
The ground truth for these rows are provided in the file train_labels.csv.

In [12]:
labels.head(5)

In [6]:
f, ax = plt.subplots(1, 2, figsize = (15, 7))
labels_names = [f"{p:.2f}%" for p in labels.value_counts() / labels.value_counts().sum() * 100]
labels.value_counts().plot(kind='pie', ax=ax[0], labels=labels_names, colors=("r", "b"), ylabel="label")
labels.value_counts().plot(kind='bar', ax=ax[1], color=("r", "b"))
plt.show()

#### Submissions files
Each file name in the submissions folder corresponds to the logloss score of the the first half of the prediction rows (20k rows) against the ground truth labels in that file. This is effectively the "training" set.

(트레이닝 세트)

In [7]:
s0 = pd.read_csv(SUBMISSION_FILES_PATH / subs[0], index_col='id')

In [8]:
'''
log_loss 
모델 성능 평가 시 사용 가능한 지표
분류(Classification) 모델 평가시 사용합니다.
값이 작을수록 좋은 모델
'''

score = log_loss(labels, s0.loc[gt_ids])

# Notice the score of the labeled rows matches the file name
print(subs[0],' log_loss:', f'{score:.10f}')

0.6222863195.csv  log_loss: 0.6222863195


In [35]:
subs[0:10]

#### Loading all submission files
we are going to load all submission files into a dataframe with final shape 40k x 5k. Each Submission file will go into a column of the dataframe

We also calculate the ROC-AUC for each submission file in order to use this metric later

In [10]:
X_train_orig = np.zeros((s0.shape[0], len(subs))) # (5000, 40000)
roc_auc_scores = dict()

for i, name in tqdm(enumerate(subs)):
    
    # submission_files에 파일들을 하나씩 읽어와서 X_train_orig에 첫번째 row에 저장
    sub = pd.read_csv(SUBMISSION_FILES_PATH / name, index_col='id')
    X_train_orig[:, i] = sub.pred.values
    
    # roc_auc_score 정확도 평가
    # ROC Curve의 면적, AUC의 값이 1일 수록 좋은 모델이라 평가
    # (실제 정답 리스트(정수), 1일 확률이 담긴 리스트(실수)) 이렇게 연산한 결과가 1에 가까울 정도로 높은 수치가 나옴
    # labels.label 20k개, X_train_orig에서 i columns의 0~20k까지의 row 값
    auc_score = roc_auc_score(labels.label[0:20000], X_train_orig[0:20000, i])
    
    # 연산된 수치값을 roc_auc_scores에 저장
    roc_auc_scores[name] = auc_score
    
# X_train_orig으로 만들어진 array?로 DataFrame 생성
X_train_orig = pd.DataFrame(X_train_orig, columns=subs)

5000it [06:09, 13.54it/s]


In [11]:
X_train_orig.head(10)

#### Removing Strange submission files
we are going to remove strange submission files with values >1 or <0 As result we remove 108 submission files

Instead of removing strange submission files we try to apply

In [12]:
'''
X_train_orig = X_train_orig.loc[:, X_train_orig.max()<=1]
X_train_orig = X_train_orig.loc[:, X_train_orig.min()>=0]

numpy.clip(array, min, max)
array 내의 element들에 대해서
min 값 보다 작은 값들을 min값으로 바꿔주고
max 값 보다 큰 값들을 max값으로 바꿔주는 함수.
'''

X_train_orig = X_train_orig.clip(0, 1)

# 2 variables removed since they were low-information variables
# 0.6933054832.csv, 0.6933472206.csv cloumns 값이 전부 동일
DROP_LOW_INFO_FEATURES = ['0.6933054832.csv', '0.6933472206.csv']

# 제거후 X_train_orig에 적용 
X_train_orig = X_train_orig[list(set(X_train_orig.columns) - set(DROP_LOW_INFO_FEATURES))]

#### Calibration of Train set (Train set 보정)
Our train set is composed of several models' predictions output, so it's a good idea to calibrate their output before training a blended model with these  
(현재 Train set는 여러 모델의 예측 값으로 구성되어 있어 혼합 모델을 training 전 보정이 필요)

ref. https://www.kaggle.com/competitions/tabular-playground-series-nov-2022/discussion/363778

##### sklearn CalibrationDisplay

교정 곡선(신뢰도 다이어그램이라고도 함) 시각화
실제 레이블과 예측 확률을 사용하여 보정(교졍) 곡선을 그립니다.

---

**Methods**

**from_estimator(estimator, X, y, *[, n_bins, ...]):** Plot calibration curve using a binary classifier and data.

**from_predictions(y_true, y_prob, *[, ...]):** Plot calibration curve using true labels and predicted probabilities.

**plot(*[, ax, name, ref_line]):** Plot visualization.

---

**Parameters**


**'X':** {array-like, sparse matrix} of shape (n_samples, n_features)
    Input values.
    
**'y':** array-like of shape (n_samples,)
    Binary target values.
    
**'n_bins':** int, default=5
    Number of bins to discretize the [0, 1] interval into when calculating the calibration curve. 
    A bigger number requires more data.
    
**'strategy':** {‘uniform’, ‘quantile’}, default=’uniform’
    Strategy used to define the widths of the bins.
    
    - 'uniform': The bins have identical widths.
    - 'quantile': The bins have the same number of samples and depend on predicted probabilities.
        
**'pos_label':** str or int, default=None
    The positive class when computing the calibration curve. 
    By default, estimators.classes_[1] is considered as the positive class.
    
**'name':** str, default=None
    Name for labeling curve. If None, the name of the estimator is used.
    
**'ref_line':** bool, default=True
    If True, plots a reference line representing a perfectly calibrated classifier.
    
**'ax':** matplotlib axes, default=None
    Axes object to plot on. If None, a new figure and axes is created.
    
---

ref: https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibrationDisplay.html

In [13]:
# example of an uncalibrated model prediction on train set

X_train_orig[:20000]['0.6222863195.csv']
X_train_orig[:]['0.6223807245.csv'].plot(kind='hist', bins=100)
    
CalibrationDisplay.from_predictions(
    labels.label[0:20000], 
    X_train_orig[0:20000]['0.6223807245.csv'], 
    n_bins=20,
    strategy='quantile', 
    color='#ffd700')
plt.show()

##### sklearn sotonicRegression 등회귀

■ 등회귀는 언제 사용되나?
등회귀는 통계적 추론에 사용된다. 예를 들어, x의 순서에 따라 y값이 증가할 것으로 예상되는 데이터 셋의 경우, 등회귀식을 적용할 수 있다. 
분류(classification) 문제에서는 모델의 예측값을 보정(calibration)할 때도 사용된다. 

---

**Methods**

**fit(X, y[, sample_weight]):** Fit the model using X, y as training data.

**fit_transform(X[, y]):** Fit to data, then transform it.

**get_feature_names_out([input_features]):** Get output feature names for transformation.

**get_params([deep]):** Get parameters for this estimator.

**predict(T):** Predict new data by linear interpolation.

**score(X, y[, sample_weight]):** Return the coefficient of determination of the prediction.

**set_params(**params):** Set the parameters of this estimator.

**transform(T):** Transform new data by linear interpolation.

---

**Parameters**

**y_minfloat, default=None:**
Lower bound on the lowest predicted value (the minimum value may still be higher). If not set, defaults to -inf.

**y_maxfloat, default=None:**
Upper bound on the highest predicted value (the maximum may still be lower). If not set, defaults to +inf.

**increasingbool or ‘auto’, default=True:**
Determines whether the predictions should be constrained to increase or decrease with X. ‘auto’ will decide based on the Spearman correlation estimate’s sign.

**out_of_bounds{‘nan’, ‘clip’, ‘raise’}, default=’nan’:**
Handles how X values outside of the training domain are handled during prediction.

- ‘nan’, predictions will be NaN.

- ‘clip’, predictions will be set to the value corresponding to the nearest train interval endpoint.

- ‘raise’, a ValueError is raised.

---

ref: https://scikit-learn.org/stable/modules/generated/sklearn.isotonic.IsotonicRegression.html

In [14]:
# train set calibration 
# ref. https://www.kaggle.com/competitions/tabular-playground-series-nov-2022/discussion/363778

roc_auc_scores_calibrated = {}
X_train_calibrated = X_train_orig.copy()

for i, c in tqdm(enumerate(X_train_orig.columns)):
    
    # 길이가 40k인 list 생성
    x_model_calibration = np.zeros(40000)
    
    # 등회귀 모델 생성
    model_calibration = IsotonicRegression(out_of_bounds='clip')
    
    # train 데이터를 fit_transform()을 하여 범위를 맞추고 학습후? 변환 및 저장 
    # clip으로 min=0.001, max=0.999로 변환
    x_model_calibration[:20000] = model_calibration.fit_transform(
        X_train_orig[:20000][c], labels.label).clip(0.001, 0.999)
    
    # 나머지 부분도 변환 후 저장
    # clip으로 min=0.001, max=0.999로 변환
    x_model_calibration[20000:] = model_calibration.transform(
        X_train_orig[20000:][c]).clip(0.001, 0.999)
    
    # 결과 값 X_train_calibrated dict에 저장
    X_train_calibrated[c] = x_model_calibration
    
    # roc_auc_score 정확도 평가
    # ROC Curve의 면적, AUC의 값이 1일 수록 좋은 모델이라 평가
    auc_score = roc_auc_score(labels.label[0:20000], x_model_calibration[0:20000])
    
    # 평가된 값을 roc_auc_scores_calibrated dict에 저장
    roc_auc_scores_calibrated[c] = auc_score
    
    # 시각화
    pd.DataFrame(X_train_calibrated[:20000][X_train_calibrated.columns[0]]).plot(
        kind='hist', bins=100)
    
    # 실제 레이블과 예측 확률을 사용하여 보정(교정) 곡선 시각화.
    CalibrationDisplay.from_predictions(
        labels.label[0:20000], 
        X_train_calibrated[:20000][X_train_calibrated.columns[0]], 
        n_bins=20,                                
        strategy='quantile', 
        color='#ffd700')
    
plt.show()

In [15]:
print(roc_auc_scores_calibrated)

##### Dimensionality Reduction with ROC-AUC? Why not!

---

##### Idea for Feature Selection using ROC-AUC

A bad classifier can be identified by the ROC curve which looks very similar, if not identical, to the diagonal of the graph, representing the performance of a purely random classifier; ROC-AUC scores close to 0.5 are considered near-random results.  

---

Starting from the concepts behind ROC-AUC explained above, we are going to use only submissions where the ROC-AUC is major of a threshold  
ROC-AUC가 주요 임계값만 사용해서 제출한다?

In [16]:
AUC_TH = 0.787
fig = plt.figure(figsize=(30, 5))
fig.suptitle("ROC-AUC Distribution")
out = plt.hist(roc_auc_scores_calibrated.values(), bins=200, color=("b"))
plt.plot([AUC_TH, AUC_TH], [0, 500], linestyle='dotted', c="r")
plt.show()

In [17]:
SELECTED_FEATURES = []
TOP_LOG_LOSS_TH = 500


for k, v in roc_auc_scores_calibrated.items():    
    
    # AUC_TH = 0.787
    if v >= AUC_TH:
        if k in X_train_orig.columns:
            SELECTED_FEATURES.append(k)
            
print(len(SELECTED_FEATURES), SELECTED_FEATURES[:TOP_LOG_LOSS_TH])

# X_train_calibrated (등회귀로 보정된 값이 저장된 dict)에 AUC_TH가 0.787 같거나 큰 key 값만 뽑아서(최대 500개) 저장
X_train_reduced = X_train_calibrated[SELECTED_FEATURES[:TOP_LOG_LOSS_TH]]
print(X_train_reduced.shape)

##### Feature Selection with MRMR (optional)
Maximum Relevance — Minimum Redundancy” (aka MRMR) is an algorithm used by Uber’s machine learning platform for finding the “minimal-optimal” subset of features.

- Enabling FEATURE_SELECTION_ENABLED the train will be done with the features selection algorithm
- Disabling FEATURE_SELECTION_ENABLED the train will be done with all features availables in the train set (all columns)

##### mrmr_classif

---

mRMR은 filter method의 한 방법으로써, 두 가지 요소를 고려합니다. 바로, Y를 잘 예측하는 변수이면서, X들과도 중복성이 적은 변수들을 선택합니다. 이 때, Y와의 상관성과 X들과의 중복성은 일반적으로 피어슨 상관계수와 Mutual information으로 측정할 수 있습니다.

---

ref: https://github.com/smazzanti/mrmr

In [18]:
FEATURE_SELECTION_ENABLED = True
# FEATURE SELECTION MRMR
def feature_selection(X, y, k):
    if not FEATURE_SELECTION_ENABLED:
        return X.columns
    
    # mrmr로 Feature Selection
    # k Parameter는 받아볼 결과 갯수?
    out = mrmr_classif(X, y, k)
    print("Features selection:", out)
    return out

if FEATURE_SELECTION_ENABLED:
    FEATURES_SELECTED = feature_selection(X_train_reduced[0:20000], labels.label, 50)
    X_train_reduced = X_train_reduced[FEATURES_SELECTED]

In [19]:
X_train_reduced

###### Feature Engineering

---

**Adding Unsupervised new feature**

we try to add an unsupervised new feature calculated with kmeans and others as combination of features and mean

In [20]:
X_train_reduced['cluster'] = KMeans(n_clusters=5, random_state=seed).fit_predict(X_train_reduced)
X_train_reduced['mean'] = X_train_reduced.mean(axis=1)

# compose a new feature as combination of two best features
X_train_reduced['compose_feature'] = (
    X_train_reduced['0.6778730537.csv'] + X_train_reduced['0.6702783631.csv']) / X_train_reduced['mean']

In [20]:
X_train_reduced['cluster']

In [21]:
X_train_reduced['mean']

In [22]:
X_train_reduced['compose_feature']

##### Blending/Stacking Model

StratifiedKFold을 이용한 학습 데이터 분류 (교차검증)

In [25]:
FOLDS = 5
k_fold = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed)
y = labels
X = X_train_reduced[0:20000]
X_test = X_train_reduced[20000:]

In [23]:
y

In [24]:
X

In [25]:
X_test

##### RandomSearchCV

To find the best hyperparameters, we will use the RandomSearchCV. Random search is a technique more faster than GridSearchCV which calculates all possible combinations

RandomSearchCV를 이용해 best hyperparameters 찾기

In [30]:
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, test_size = 0.01, shuffle=True, random_state= seed, stratify=y)

X_train = X_train.reset_index().drop("index", axis=1)

X_validation = X_validation.reset_index().drop("index", axis=1)

y_train = y_train.reset_index()['label']

y_validation = y_validation.reset_index()['label']

In [26]:
X_train

In [27]:
X_validation

In [28]:
y_train

In [29]:
y_validation

In [35]:
fit_params = {
    'eval_metric': 'binary_logloss', 
    'eval_set': [(X_validation, y_validation)],   
    'verbose': 0
}

param_test = {
  'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.1, 0.2, 0.3, 0.4],
  'n_estimators': [100, 200, 300, 400, 500, 600, 800, 1000, 1500, 2000],
  'num_leaves': sp_randint(6, 50, 100), 
  'min_child_samples': sp_randint(100, 500), 
  'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
  'subsample': sp_uniform(loc=0.2, scale=0.8), 
  'max_depth': [-1, 1, 2, 3, 4, 5, 6, 7],
  'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
  'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
  'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
}

In [36]:
lgbm_rs = LGBMClassifier(metric = 'binary_logloss', 
    random_state = seed, 
    silent = True, 
    n_jobs = -1
)

random_search = RandomizedSearchCV(
    estimator = lgbm_rs, 
    param_distributions = param_test,     
    scoring = 'neg_log_loss',
    cv = k_fold,
    refit = True,
    random_state = seed,
    verbose = 100, 
    n_iter = 50
)

opt_parameters = {
    'colsample_bytree': 0.45066268483824545,
    'learning_rate': 0.02,
    'max_depth': 5,
    'min_child_samples': 285,
    'min_child_weight': 0.01,
    'n_estimators': 300,
    'num_leaves': 116,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'subsample': 0.532329735064063
}

RS_FIT = False # enable it to use RandomSearchCV

if RS_FIT:
    random_search.fit(X, y, **fit_params)
    opt_parameters = random_search.best_params_

In [30]:
opt_parameters

##### Hybrid Model Class
The following class permit us to encapsulate the logic of ensembling N models making a weighted average of their predictions (Soft-Voting)

We are going to choose several models in order to ensemble them and try to get better results

In [38]:
class EnsembleHybrid:
   def __init__(self, models=[], weights=[]):
       self.models = models
       self.weights = weights

   def fit(self, X, y):
       # Train models
       for m in self.models:
           print(f"Training {m}...")
           m.fit(X, y)

   def predict_proba(self, X_test):
       y_pred = pd.Series(np.zeros(X_test.shape[0]), index=X_test.index)
       for i, m in enumerate(self.models):
           y_pred += pd.Series(m.predict_proba(X_test)[:,1], index=X_test.index) * self.weights[i]
       return y_pred

In [31]:
def folds_model(debug=True):
    # several models in order to stacking
    lgbm = LGBMClassifier(
        **opt_parameters, 
        objective='binary', 
        silent=True,
        random_state=seed,
        metric='binary_logloss'
    )
    
#     # 사용하지 않는 것으로 보임
#     lgbm_t = LGBMClassifier(
#         objective='binary', 
#         silent=True,
#         random_state=seed,
#         metric='binary_logloss', 
#         bagging_fraction=0.5, 
#         bagging_freq=3, 
#         boosting_type='gbdt',
#         class_weight=None, 
#         colsample_bytree=1.0, 
#         feature_fraction=0.7,
#         importance_type='split', 
#         learning_rate=0.001, 
#         max_depth=-1,
#         min_child_samples=6, 
#         min_child_weight=0.001, 
#         min_split_gain=0.9,
#         n_estimators=140, 
#         n_jobs=-1, 
#         num_leaves=60, 
#         reg_alpha=0.4, 
#         reg_lambda=1e-07, 
#         subsample=1.0, 
#         subsample_for_bin=200000, 
#         subsample_freq=0
#     )

    cat_boost = CatBoostClassifier(
        random_seed=seed,
        eval_metric='Logloss',
        logging_level='Silent',
        learning_rate=0.05,
        iterations=200
    )
    
#     # 사용하지 않는 것으로 보임
#     cat_boost_t = CatBoostClassifier(
#         random_seed=seed,
#         eval_metric='Logloss',
#         logging_level='Silent',
#         learning_rate=0.05,
#         iterations=200
#     )

#     # 사용하지 않는 것으로 보임
#     xgbm = XGBClassifier(
#         objective='binary:logistic',
#         random_state=seed,
#         learning_rate=0.1,
#         n_estimators=100,
#         max_depth=8, 
#         #tree_method='gpu_hist'
#     )
    
#     # 사용하지 않는 것으로 보임
#     xgbm_t = XGBClassifier(
#         objective='binary:logistic',
#         random_state=seed,
#         learning_rate=0.1,
#         n_estimators=100,
#         max_depth=8, 
#         #tree_method='gpu_hist'
#     )
    
    models = [lgbm, cat_boost]
    weights=[0.2, 0.8]
    Y_validations, ensemble_val_preds, ensemble_test_preds, scores = [],[],[],[]
    
    # Kfold.split()으로 반환된 인덱스를 이용, 학습용 검증용 데이터 구성
    for fold, (train_idx, val_idx) in enumerate(k_fold.split(X, y)):
        
        if debug:
            print(f'\nFold {fold+1}')
            
        X_fold_val, Y_fold_val = X.iloc[val_idx, :], y.label[val_idx]
        X_fold_train, Y_fold_train = X.iloc[train_idx, :], y.label[train_idx]
        
        if debug:
            print(f'Train shape: {X_fold_train.shape}, {Y_fold_train.shape}, Valid shape: {X_fold_val.shape}, {Y_fold_val.shape}')

        # ensemble class 생성
        ensemble_model = EnsembleHybrid(models=models, weights=weights)
        
        # model fit
        ensemble_model.fit(X_fold_train, Y_fold_train)
        
        # model predict
        model_prob = ensemble_model.predict_proba(X_fold_val)        

        ensemble_prob = ensemble_model.predict_proba(X_fold_val)
        
        Y_validations.append(Y_fold_val)
        
        ensemble_val_preds.append(ensemble_prob)
        
        ensemble_test_preds.append(ensemble_model.predict_proba(X_test))        

        score = log_loss(Y_fold_val, ensemble_prob)
        scores.append(score)
        
        if debug:
            print(f'Fold {fold+1} Validation Score = {score:.4f}')

        del X_fold_train, Y_fold_train, X_fold_val, Y_fold_val 
        
    mix_score = sum(scores)/FOLDS
    
    if debug:
        print('Total Score (Mixing Folds Predictions) = {:.4f}'.format(mix_score))
        
    return mix_score, ensemble_test_preds

score, ensemble_test_preds = folds_model()

In [32]:
ensemble_test_preds

In [42]:
folds_blend = np.zeros(X_test.shape[0])

for j in range(FOLDS):
    folds_blend += ensemble_test_preds[j]
    
folds_blend = folds_blend / FOLDS
submission['pred'] = folds_blend
submission.to_csv('final_submission_folds_mix.csv')

In [33]:
submission['pred']

In [34]:
final = pd.read_csv(BASE_DIR / 'shc/final_submission_folds_mix.csv')
final